# DeepLob 系列论文

- https://arxiv.org/abs/2403.09267 # Deep Limit Order Book Forecasting
- https://arxiv.org/abs/1811.10041 # BDLOB: Bayesian Deep Convolutional Neural Networks for Limit Order Books
- https://arxiv.org/abs/1803.06917 # Universal features of price formation in financial markets: perspectives from Deep Learning
- https://arxiv.org/abs/2101.07107 # Deep Reinforcement Learning for Active High Frequency Trading
- https://arxiv.org/abs/1808.03668 # DeepLOB: Deep Convolutional Neural Networks for Limit Order Books
- https://ar5iv.labs.arxiv.org/html/2003.00130 # Transformers for limit order books
- https://arxiv.org/html/2303.16532v2 # Graph Portfolio: High-Frequency Factor Predictor via Heterogeneous Continual GNNs
- https://arxiv.org/abs/2112.08534 # Trading with the Momentum Transformer: An Intelligent and Interpretable Architecture

本项目尝试复现最简单的 CNN + LSTM 模型

In [ ]:
import gc
import os

import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from sklearn.metrics import classification_report
from sklearn import preprocessing

# 避免显存不足
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# 我不知道这个操作有啥用，但是很多项目都有，我就加上了
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
   try:
       for gpu in gpus:
           tf.config.experimental.set_memory_growth(gpu, True)
       logical_gpus = tf.config.experimental.list_logical_devices('GPU')
       print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
   except RuntimeError as e:
       print(e)

本项目只需要 limit order book 数据

是否需要处理行情隔天开盘时的跳空？我觉得是需要的，但是所有的paper都不处理，可能搞学术的不想干脏活累活。

In [ ]:
# 行情数据用 tqsdk 下载的，参考download.py
data = pd.read_csv('rb2501_tick.csv')
data.rename(columns=lambda x: x.replace('SHFE.rb2501.', ''), inplace=True)
col = ['bid_price1', 'bid_volume1', 'ask_price1', 'ask_volume1', 'bid_price2', 'bid_volume2', 'ask_price2', 'ask_volume2', 'bid_price3', 'bid_volume3', 'ask_price3', 'ask_volume3', 'bid_price4', 'bid_volume4', 'ask_price4', 'ask_volume4', 'bid_price5', 'bid_volume5', 'ask_price5', 'ask_volume5']
data = data[col]
data = data.dropna()
gc.collect()
print(data.shape)

行情label算法，将未来行情分为
- 0 上涨
- 1 下跌
- 2 平稳

样本中3个类别是否平衡，决定模型loss函数的选择

类别极端不平衡时，需要考虑使用focal loss损失函数

！我觉得label算法对模型的影响非常大，值得深入思考构造更巧妙的指标

In [3]:
# 调参的地方
spread            = 0.003  # 调节价差阈值，参考螺纹钢行情，价格变动10跳，均价3000，10/3000 = 0.003
trade_cost        = 1.002  # 调节交易成本，参考螺纹钢行情，手续费3块钱，买卖一手 3*2/3000 = 0.002
look_ahead_period = 120*5  # 未来n个tick

short_threhold_low   = 1 - spread*2  # 做空方向止盈价 
short_threshold_high = 1 + spread*1  # 做空方向止损价
long_threshold_high  = 1 + spread*2  # 做多方向止盈价
long_threshold_low   = 1 - spread*1  # 做多方向止损价

def generate_trade_signals(bid_price: pd.Series, ask_price: pd.Series) -> pd.Series:
    '''
    根据未来价格变化大小，生成交易信号
    信号:
    - 0: 做空
    - 1: 做多
    - 2: 不操作
    '''
    signals = []

    for index in range(int(len(bid_price) - look_ahead_period)):
        # 计算当前做空后，未来的资产变化
        if min(ask_price[index: index + look_ahead_period]) <= short_threhold_low * bid_price[index] and max(
                ask_price[index: index + look_ahead_period]) >= short_threshold_high * bid_price[index]:
            # 未来行情触发止盈止损，先触发止盈，则获利
            if np.where(ask_price[index: index + look_ahead_period] <= short_threhold_low * bid_price[index])[0][0] < \
                    np.where(ask_price[index: index + look_ahead_period] >= short_threshold_high * bid_price[index])[0][0]:
                short_profit = 2 - short_threhold_low * trade_cost
            # 未来行情触发止盈止损，先触发止损，则亏损
            else:
                short_profit = 2 - short_threshold_high * trade_cost
        # 未来行情只触发止盈，获利
        elif min(ask_price[index: index + look_ahead_period]) <= short_threhold_low * bid_price[index]:
            short_profit = 2 - short_threhold_low * trade_cost
        # 未来行情只触发止损，亏损
        elif max(ask_price[index: index + look_ahead_period]) >= short_threshold_high * bid_price[index]:
            short_profit = 2 - short_threshold_high * trade_cost
        # 没有触发止盈止损，计算下最优收益，和下面的做多方向比窘，用来判断市场方向
        else: 
            short_profit = (2 * bid_price[index] - min(ask_price[index: index + look_ahead_period]) * trade_cost) / bid_price[index]

        # 做多方向
        if max(bid_price[index: index + look_ahead_period]) >= long_threshold_high * ask_price[index] and min(
                bid_price[index: index + look_ahead_period]) <= long_threshold_low * ask_price[index]:
            if np.where(bid_price[index: index + look_ahead_period] >= long_threshold_high * ask_price[index])[0][0] < \
                    np.where(bid_price[index: index + look_ahead_period] <= long_threshold_low * ask_price[index])[0][0]:
                long_profit = long_threshold_high / trade_cost
            else:
                long_profit = long_threshold_low / trade_cost
        elif max(bid_price[index: index + look_ahead_period]) >= long_threshold_high * ask_price[index]:
            long_profit = long_threshold_high / trade_cost
        elif min(bid_price[index: index + look_ahead_period]) <= long_threshold_low * ask_price[index]:
            long_profit = long_threshold_low / trade_cost
        else:
            long_profit = max(bid_price[index: index + look_ahead_period])/ (ask_price[index] * trade_cost)

        # 资产变化超过阈值，生成交易信号
        signals.append(np.argmax([short_profit * int(short_profit > 1+spread), long_profit * int(long_profit > 1+spread), 1]))
    return pd.Series(signals)

In [4]:

def get_label(data: pd.DataFrame) -> pd.Series:
    bid_price = data.iloc[:, 0]
    ask_price = data.iloc[:, 2]
    labels = generate_trade_signals(bid_price, ask_price)
    return labels


length = data.shape[0]

train_data = data.iloc[1:int(length*0.8),:].reset_index(drop=True)
train_label = get_label(train_data)


valid_data = data.iloc[int(length*0.8):int(length*0.9),:].reset_index(drop=True)
valid_label = get_label(valid_data)


test_data = data.iloc[int(length*0.9):int(length),:].reset_index(drop=True)
test_label = get_label(test_data)


# 归一化，注意要用train_data的均值和方差，对valid_data和test_data进行归一化，否则会引入未来信息
ss = preprocessing.StandardScaler().fit(train_data)
train_data = pd.DataFrame(ss.transform(train_data)).values[:-look_ahead_period,:]
valid_data = pd.DataFrame(ss.transform(valid_data)).values[:-look_ahead_period,:]
test_data  = pd.DataFrame(ss.transform(test_data)).values[:-look_ahead_period,:]


# label 过程非常慢，缓存数据，避免重复计算

np.save('train_data.npy', train_data)
np.save('train_label.npy', train_label)
np.save('valid_data.npy', valid_data)
np.save('valid_label.npy', valid_label)
np.save('test_data.npy', test_data)
np.save('test_label.npy', test_label)


In [ ]:
train_data = np.load('train_data.npy')
train_label = np.load('train_label.npy')
print(train_label.value_counts())
changes = (train_label != train_label.shift()).sum()
print(f"Train label changes: {changes}")

valid_data = np.load('valid_data.npy')
valid_label = np.load('valid_label.npy')
print(valid_label.value_counts())
changes = (valid_label != valid_label.shift()).sum()
print(f"Valid label changes: {changes}")

test_data = np.load('test_data.npy')
test_label = np.load('test_label.npy')
print(test_label.value_counts())
changes = (test_label != test_label.shift()).sum()
print(f"Test label changes: {changes}")

In [ ]:
def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)
    dY = np.array(Y)
    dataY = dY[T - 1:N]
    dataX = np.zeros((N - T + 1, T, D),dtype='float16')
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]
    return dataX.reshape(dataX.shape + (1,)), dataY

# 用过去T个tick的数据预测，这个参数可以调整，对gpu显存大小有影响，8G显存最多也就60了
T = 60
trainX_CNN, trainY_CNN = data_classification(train_data, train_label, T)
trainY_CNN = keras.utils.to_categorical(trainY_CNN, 3)

validX_CNN, validY_CNN = data_classification(valid_data, valid_label, T)
validY_CNN = keras.utils.to_categorical(validY_CNN, 3)
testX_CNN, testY_CNN = data_classification(test_data, test_label, T)

testY_CNN = keras.utils.to_categorical(testY_CNN, 3)
del train_data, train_label, valid_data, valid_label,test_data, test_label
del data
gc.collect()

In [7]:
def create_deeplob(T, NF):
    input_lmd = keras.layers.Input(shape=(T, NF, 1))

    node = 16
    negative_slope = 0.01
    # build the convolutional block
    conv_first1 = keras.layers.Conv2D(node, (1, 2), strides=(1, 2))(input_lmd)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)
    conv_first1 = keras.layers.Conv2D(node, (5, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)
    
    conv_first1 = keras.layers.Conv2D(node, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)
    conv_first1 = keras.layers.Conv2D(node, (5, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)

    conv_first1 = keras.layers.Conv2D(node, (1, 5))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)
    conv_first1 = keras.layers.Conv2D(node, (5, 1), padding='same')(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(conv_first1)
    
    node2 = 16
    # build the inception module
    convsecond_1 = keras.layers.Conv2D(node2, (1, 1), padding='same')(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(convsecond_1)
    convsecond_1 = keras.layers.Conv2D(node2, (3, 1), padding='same')(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(negative_slope=negative_slope)(convsecond_1)

    convsecond_2 = keras.layers.Conv2D(node2, (1, 1), padding='same')(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(negative_slope=negative_slope)(convsecond_2)
    convsecond_2 = keras.layers.Conv2D(node2, (5, 1), padding='same')(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(negative_slope=negative_slope)(convsecond_2)

    convsecond_3 = keras.layers.MaxPooling2D((3, 1), strides=(1, 1), padding='same')(conv_first1)
    convsecond_3 = keras.layers.Conv2D(node2, (1, 1), padding='same')(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(negative_slope=negative_slope)(convsecond_3)

    convsecond_output = keras.layers.concatenate([convsecond_1, convsecond_2, convsecond_3], axis=3)
    conv_reshape = keras.layers.Reshape((int(convsecond_output.shape[1]), int(convsecond_output.shape[3])))(convsecond_output)

    # build the last LSTM layer
    # 这个参数可以调整，对gpu显存大小有影响，8G显存最多也就32了
    # 我怎么调这个参数模型效果都差不多，不知道为什么
    number_of_lstm = 8
    conv_lstm = keras.layers.LSTM(number_of_lstm)(conv_reshape)

    # build the output layer
    out = keras.layers.Dense(3, activation='softmax')(conv_lstm)
    model = keras.models.Model(inputs=input_lmd, outputs=out)
    adam = keras.optimizers.Adam()
    loss = keras.losses.CategoricalFocalCrossentropy(
        alpha=1,  # 平衡因子，用于平衡正负样本
        gamma=2  # 焦点因子，用于关注难分类的样本
    ),
    model.compile(optimizer=adam, loss=loss, metrics=['accuracy'])
    # model.compile(optimizer=adam, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

    return model

deeplob = create_deeplob(T, 20)
# deeplob.summary()


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='auto')
checkpoint_filepath = './model_check/sandra.weights.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                            save_weights_only=True,
                                                            monitor='val_loss',
                                                            mode='auto',
                                                            save_best_only=True)
deeplob.fit(trainX_CNN, trainY_CNN, epochs=200, batch_size=128, verbose=2, validation_data=(validX_CNN, validY_CNN),
            callbacks=[model_checkpoint_callback, early_stopping])  
      

In [ ]:
del trainX_CNN, trainY_CNN, validX_CNN, validY_CNN
gc.collect()
# evaluate the model
deeplob.load_weights(checkpoint_filepath)
predictions = deeplob.predict(testX_CNN)
results = keras.utils.to_categorical(np.argmax(predictions, axis=1), 3)
print(classification_report(testY_CNN, results, target_names=['0', '1', '2']))

print("Evaluate")
result = deeplob.evaluate(testX_CNN, testY_CNN, verbose=0)
print(dict(zip(deeplob.metrics_names, result)))
